# Schematic

A schematic is a graph representation of your circuit.

For complex circuits, a schematic allows you to create symbols and hierarchy levels to represent your circuit.

Having a schematic allows you to also ensure that your layout matches you schematic (design intent).

There are many schematic capture tools out there:

- Qucs-s: for RF.
- Xschem: for analog circuits.
- Lumerical interconnect: for photonic circuits.

These tools allow you to both create schematics with your mouse or by code.

gdsfactory also allows you to create complex Schematics directly from python with a very simple interface.

In [ ]:
import gdsfactory as gf
import gdsfactory.schematic as gt
import yaml

In [ ]:
c = gf.c.mzi()
c.plot_netlist_graphviz(interactive=False)

In [ ]:
c.plot_netlist()

Lets create a MZI lattice of 3 elements.

In [ ]:
s = gt.Schematic()
s.add_instance("mzi1", gt.Instance(component=gf.c.mzi(delta_length=10)))
s.add_instance("mzi2", gt.Instance(component=gf.c.mzi(delta_length=100)))
s.add_instance("mzi3", gt.Instance(component=gf.c.mzi(delta_length=200)))
s.add_placement("mzi1", gt.Placement(x=000, y=0))
s.add_placement("mzi2", gt.Placement(x=100, y=100))
s.add_placement("mzi3", gt.Placement(x=200, y=0))
s.add_net(gt.Net(p1="mzi1,o2", p2="mzi2,o1"))
s.add_net(gt.Net(p1="mzi2,o2", p2="mzi3,o1"))
g = s.plot_graphviz()

You can also create a splitter tree.

In [ ]:
s = gt.Schematic()
s.add_instance("s11", gt.Instance(component=gf.c.mmi1x2()))
s.add_instance("s21", gt.Instance(component=gf.c.mmi1x2()))
s.add_instance("s22", gt.Instance(component=gf.c.mmi1x2()))
s.add_placement("s11", gt.Placement(x=000, y=0))
s.add_placement("s21", gt.Placement(x=100, y=+50))
s.add_placement("s22", gt.Placement(x=100, y=-50))
s.add_net(gt.Net(p1="s11,o2", p2="s21,o1"))
s.add_net(gt.Net(p1="s11,o3", p2="s22,o1"))
g = s.plot_graphviz()

The nice thing is that you can abstract it to have as many levels as you need.

In [ ]:
splitter = gf.components.mmi1x2()
n = 3
dx = 100
dy = 100
s = gt.Schematic()

for col in range(n):
    rows = 2**col
    for row in range(rows):
        s.add_instance(f"s{col}{row}", gt.Instance(component=splitter))
        s.add_placement(
            f"s{col}{row}", gt.Placement(x=col * dx, y=(row - rows / 2) * dy)
        )
        if col < n - 1:
            s.add_net(gt.Net(p1=f"s{col}{row},o2", p2=f"s{col+1}{2*row},o1"))
            s.add_net(gt.Net(p1=f"s{col}{row},o3", p2=f"s{col+1}{2*row+1},o1"))


g = s.plot_graphviz()

In [ ]:
splitter = gf.components.mmi1x2()
n = 5
dx = 100
dy = 100
s = gt.Schematic()

for col in range(n):
    rows = 2**col
    for row in range(rows):
        s.add_instance(f"s{col}{row}", gt.Instance(component=splitter))
        s.add_placement(
            f"s{col}{row}", gt.Placement(x=col * dx, y=(row - rows / 2) * dy)
        )
        if col < n - 1:
            s.add_net(gt.Net(p1=f"s{col}{row},o2", p2=f"s{col+1}{2*row},o1"))
            s.add_net(gt.Net(p1=f"s{col}{row},o3", p2=f"s{col+1}{2*row+1},o1"))


g = s.plot_graphviz()

In [ ]:
dict(s.netlist)

In [ ]:
yaml_component = yaml.dump(s.netlist.model_dump(exclude_none=True))
print(yaml_component)

In [ ]:
yaml.dump(s.netlist.model_dump(exclude_none=True), open("schematic.yaml", "w"))

## Python routing

In [ ]:
n = 2**3
splitter = gf.components.splitter_tree(noutputs=n, spacing=(50, 50))
dbr_array = gf.components.array(
    component=gf.c.dbr, rows=n, columns=1, spacing=(0, 3), centered=True
)
s = gt.Schematic()
s.add_instance("s", gt.Instance(component=splitter))
s.add_instance("dbr", gt.Instance(component=dbr_array))
s.add_placement("s", gt.Placement(x=0, y=0))
s.add_placement("dbr", gt.Placement(x=300, y=0))

for i in range(n):
    s.add_net(
        gt.Net(
            p1=f"s,o2_2_{i+1}",
            p2=f"dbr,o1_{i+1}_1",
            name="splitter_to_dbr",
            settings=dict(radius=5, sort_ports=True, cross_section="strip"),
        )
    )

g = s.plot_graphviz()

In [ ]:
dbr_array.pprint_ports()

In [ ]:
splitter.pprint_ports()

In [ ]:
yaml.dump(s.netlist.model_dump(exclude_none=True), open("schematic.yaml", "w"))
yaml_component = yaml.dump(s.netlist.model_dump(exclude_none=True))
print(yaml_component)

In [ ]:
from gdsfactory.serialization import convert_tuples_to_lists

conf = s.netlist.model_dump(exclude_none=True)
conf = convert_tuples_to_lists(conf)
yaml_component = yaml.dump(conf)
print(yaml_component)

In [ ]:
c = gf.read.from_yaml(yaml_component)
c